In [1]:
import os
import numpy as np
import pandas as pd
import torch
import os
import json
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm
import glob
import pickle

In [2]:
DATA_DIR = '/mnt/chicm/data/open-images/relation'
DETECT_DATA_DIR = '/mnt/chicm/data/open-images/detect'

In [3]:
!ls $DATA_DIR

challenge-2019-attributes-description.csv
challenge-2019-classes-vrd.csv
challenge-2019-relationships-description.csv
challenge-2019-relationship-triplets.csv
challenge-2019-train-vrd-bbox.csv
challenge-2019-train-vrd.csv
challenge-2019-train-vrd-labels.csv
challenge-2019-validation-vrd-bbox.csv
challenge-2019-validation-vrd.csv
challenge-2019-validation-vrd-labels.csv
classes_is_42.csv
negative_tuples_0804.csv
negative_tuples.csv
top_classes_42.csv
top_classes.csv
VRD_sample_submission.csv


In [9]:
df_test = pd.read_csv(os.path.join(DATA_DIR, 'VRD_sample_submission.csv'))
df_test.head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.037432 /m/03bt1vf 0.549840 0.603769 0.814588...
1,763c72626bf24534,0.037432 /m/03bt1vf 0.549840 0.603769 0.814588...
2,7540c162180973b1,0.037432 /m/03bt1vf 0.549840 0.603769 0.814588...
3,c4e522dce52cb02f,0.037432 /m/03bt1vf 0.549840 0.603769 0.814588...
4,cafcd9496c808085,0.037432 /m/03bt1vf 0.549840 0.603769 0.814588...


In [10]:
df_des = pd.read_csv(os.path.join(DATA_DIR, 'challenge-2019-classes-vrd.csv'), names=['label', 'name'])
df_des.head()

,label,name
0,/m/05r5c,Piano
1,/m/01599,Beer
2,/m/01_5g,Chopsticks
3,/m/01yrx,Cat
4,/m/04_sv,Motorcycle


In [11]:
df_des.name.values

array(['Piano', 'Beer', 'Chopsticks', 'Cat', 'Motorcycle',
       'Tennis racket', 'Football', 'Mobile phone', 'Flute',
       'Tennis ball', 'Chair', 'Woman', 'Boy', 'Coffee table', 'Handbag',
       'Fork', 'Girl', 'Coffee cup', 'Violin', 'Backpack', 'Knife',
       'Snowboard', 'Rugby ball', 'Snake', 'Oven', 'Microwave oven',
       'Man', 'Bench', 'Desk', 'Suitcase', 'Sofa bed', 'Horse',
       'Microphone', 'Table tennis racket', 'Monkey', 'Surfboard',
       'Table', 'Bottle', 'Mug', 'Wine glass', 'Drum', 'Dog', 'Guitar',
       'Ski', 'Bed', 'Spoon', 'Briefcase', 'Hamster', 'Car', 'Bicycle',
       'Camera', 'Dolphin', 'Taxi', 'Van', 'Elephant', 'Racket',
       'Pretzel'], dtype=object)

In [12]:
def get_classes():
    df_classes = pd.read_csv(os.path.join(DATA_DIR, 'classes_is_42.csv'))
    classes = df_classes['classes'].values.tolist()
    stoi = { classes[i]: i for i in range(len(classes)) }
    return classes, stoi

classes, stoi = get_classes()

In [13]:
classes[:2]

['/m/04dr76w,/m/05z87', '/m/04bcr3,/m/083vt']

In [14]:
# visualize

import numpy as np
import PIL.Image as Image
import PIL.ImageDraw as ImageDraw
import PIL.ImageFont as ImageFont

INDIGO = (75, 0, 130)


def draw_bounding_box_on_image(
    image,
    x_min,
    y_min,
    x_max,
    y_max,
    color,
    thickness=4,
    display_str_list=(),
    use_normalized_coordinates=False,
    fontsize=20
):
    draw = ImageDraw.Draw(image)
    im_width, im_height = image.size
    if use_normalized_coordinates:
        (left, right, top, bottom) = (x_min * im_width, x_max * im_width, y_min * im_height, y_max * im_height)
    else:
        (left, right, top, bottom) = (x_min, x_max, y_min, y_max)
    draw.line([(left, top), (left, bottom), (right, bottom), (right, top), (left, top)], width=thickness, fill=color)
    try:
        font = ImageFont.truetype('DejaVuSansMono.ttf', fontsize)
    except IOError:
        font = ImageFont.load_default()

    # If the total height of the display strings added to the top of the bounding
    # box exceeds the top of the image, stack the strings below the bounding box
    # instead of above.
    display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
    # Each display_str has a top and bottom margin of 0.05x.
    total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

    if top > total_display_str_height:
        text_bottom = top
    else:
        text_bottom = bottom + total_display_str_height
    # Reverse list and print from bottom to top.
    for display_str in display_str_list[::-1]:
        text_width, text_height = font.getsize(display_str)
        margin = np.ceil(0.05 * text_height)
        draw.rectangle([(left, text_bottom - text_height - 2 * margin), (left + text_width, text_bottom)], fill=color)
        draw.text((left + margin, text_bottom - text_height - margin), display_str, fill=color, font=font)
        text_bottom -= text_height - 2 * margin


def draw_bounding_boxes_on_image_array(
    image, bboxes, color=INDIGO, thickness=4, use_normalized_coordinates=False, fontsize=20
):
    image_pil = Image.fromarray(image)
    draw_bounding_boxes_on_image(image_pil, bboxes, color, thickness, use_normalized_coordinates, fontsize)
    np.copyto(image, np.array(image_pil))


def draw_bounding_boxes_on_image(
    image, bboxes, color=INDIGO, thickness=4, use_normalized_coordinates=False, fontsize=20
):
    for bbox in bboxes:
        draw_bounding_box_on_image(
            image, bbox[0], bbox[1], bbox[2], bbox[3], color, thickness, (), use_normalized_coordinates, fontsize
        )


In [15]:
classes[:5]

['/m/04dr76w,/m/05z87',
 '/m/04bcr3,/m/083vt',
 '/m/04bcr3,/m/05z87',
 '/m/01_5g,/m/083vt',
 '/m/01940j,/m/0dnr7']

In [19]:
TEST_IMG_DIR = '/mnt/chicm/data/open-images/test'
def show_preds(idx):
    plt.figure(figsize=(15, 10))
    p = get_preds(preds[idx])
    img_id = df_test.iloc[idx].ImageId
    fn = os.path.join(TEST_IMG_DIR, '{}.jpg'.format(img_id))
    img = Image.open(fn)
    draw_bounding_boxes_on_image(img, p['bboxes'])
    
    plt.imshow(img)
    labels = [classes[i].split(',')[0] for i in p['labels']]
    labels = [df_des.loc[df_des.label==x].name.values[0] for x in labels]
    #print(labels)
    print(list(zip(labels, p['scores'])))

In [16]:
show_preds(2002)

NameError: name 'show_preds' is not defined

In [23]:
import struct
import imghdr

def get_image_size(fname):
    '''Determine the image type of fhandle and return its size.
    from draco'''
    with open(fname, 'rb') as fhandle:
        head = fhandle.read(24)
        if len(head) != 24:
            raise AssertionError('imghead len != 24')
        if imghdr.what(fname) == 'png':
            check = struct.unpack('>i', head[4:8])[0]
            if check != 0x0d0a1a0a:
                raise AssertionError('png check failed')
            width, height = struct.unpack('>ii', head[16:24])
        elif imghdr.what(fname) == 'gif':
            width, height = struct.unpack('<HH', head[6:10])
        elif imghdr.what(fname) == 'jpeg':
            try:
                fhandle.seek(0) # Read 0xff next
                size = 2
                ftype = 0
                while not 0xc0 <= ftype <= 0xcf:
                    fhandle.seek(size, 1)
                    byte = fhandle.read(1)
                    while ord(byte) == 0xff:
                        byte = fhandle.read(1)
                    ftype = ord(byte)
                    size = struct.unpack('>H', fhandle.read(2))[0] - 2
                # We are at a SOFn block
                fhandle.seek(1, 1)  # Skip `precision' byte.
                height, width = struct.unpack('>HH', fhandle.read(4))
            except Exception: #IGNORE:W0703
                raise
        else:
            print(fname, imghdr.what(fname))
            #raise AssertionError('file format not supported')
            img = cv2.imread(fname)
            print(img.shape)
            height, width, _ = img.shape

        return width, height


In [24]:
df_test['h'] = df_test.ImageId.map(lambda x: get_image_size(os.path.join(TEST_IMG_DIR, '{}.jpg'.format(x)))[1])

In [25]:
df_test['w'] = df_test.ImageId.map(lambda x: get_image_size(os.path.join(TEST_IMG_DIR, '{}.jpg'.format(x)))[0])

In [26]:
df_test.head()

,ImageId,PredictionString,h,w
0,21135d5ed0e7a0b7,0.037432 /m/03bt1vf 0.549840 0.603769 0.814588...,682,1024
1,763c72626bf24534,0.037432 /m/03bt1vf 0.549840 0.603769 0.814588...,681,1024
2,7540c162180973b1,0.037432 /m/03bt1vf 0.549840 0.603769 0.814588...,682,1024
3,c4e522dce52cb02f,0.037432 /m/03bt1vf 0.549840 0.603769 0.814588...,768,1024
4,cafcd9496c808085,0.037432 /m/03bt1vf 0.549840 0.603769 0.814588...,682,1024


In [27]:
final_preds = []

for p in tqdm(preds, total=len(preds)):
    final_preds.append(get_preds(p))

100%|██████████| 99999/99999 [00:09<00:00, 10052.98it/s]


In [28]:
total_objs = 0
for p in final_preds:
    total_objs += len(p['labels'])
print(total_objs)

4670810


In [4]:
def get_pred_str(pred, w, h):
    res = []
    for label, score, bbox in zip(pred['labels'], pred['scores'], pred['bboxes']):
        #res.append(classes[label])
        label1, label2 = classes[label].split(',')
        
        res.append('{:.7f}'.format(score))
        res.append(label1)
        res.append('{:.7f}'.format(bbox[0]/w))
        res.append('{:.7f}'.format(bbox[1]/h))
        res.append('{:.7f}'.format(bbox[2]/w))
        res.append('{:.7f}'.format(bbox[3]/h))

        res.append(label2)
        res.append('{:.7f}'.format(bbox[0]/w))
        res.append('{:.7f}'.format(bbox[1]/h))
        res.append('{:.7f}'.format(bbox[2]/w))
        res.append('{:.7f}'.format(bbox[3]/h))
        res.append('is')
        
    res = [str(x) for x in res]
    return ' '.join(res)

In [5]:
#df1 = pd.read_csv('sub_is_42/sub_retina_is_42_0802_1_lb06401.csv')
#df2 = pd.read_csv('sub_is_42/sub_is_42_0726_2_lb06653.csv')
#df3 = pd.read_csv('sub_is_42/sub_x101_is_42_0807_1_lb06130.csv')
#df1.head()  # result ensemble_is_42_0808_1_3_models.csv 0.07109

In [6]:
df1 = pd.read_csv('sub_is_42/sub_retina_is_42_0801_2_lb06618.csv')
df2 = pd.read_csv('sub_is_42/sub_is_42_0726_2_lb06653.csv')
df3 = pd.read_csv('sub_is_42/sub_is_42_cas_1024_0824_lb06432.csv')
#df1.head()  # result ensemble_is_42_0808_1_3_models.csv 0.07109

In [7]:
def get_dets_by_pred_str(pred_str):
    try:
        if len(pred_str) < 1:
            return []
    except:
        print('pred_str:', pred_str)
        raise
    det_items = pred_str.split(' ')
    dets, det = [], []
    for i in range(len(det_items)):
        det.append(det_items[i])
        if (i+1) % 12 == 0:
            dets.append(det)
            det = []
    return dets

In [9]:
df1['dets'] = df1.PredictionString.fillna('').map(lambda x: get_dets_by_pred_str(x))
df2['dets'] = df2.PredictionString.fillna('').map(lambda x: get_dets_by_pred_str(x))
df3['dets'] = df3.PredictionString.fillna('').map(lambda x: get_dets_by_pred_str(x))
df1.head()

,ImageId,PredictionString,dets
0,21135d5ed0e7a0b7,0.2302950 /m/04bcr3 0.3735417 0.7281989 0.4716...,"[[0.2302950, /m/04bcr3, 0.3735417, 0.7281989, ..."
1,763c72626bf24534,0.4685133 /m/04bcr3 0.6935107 0.8204608 0.9992...,"[[0.4685133, /m/04bcr3, 0.6935107, 0.8204608, ..."
2,7540c162180973b1,0.0569300 /m/04dr76w 0.2606493 0.2071633 0.301...,"[[0.0569300, /m/04dr76w, 0.2606493, 0.2071633,..."
3,c4e522dce52cb02f,0.4203248 /m/04bcr3 0.8271465 0.6780883 0.9907...,"[[0.4203248, /m/04bcr3, 0.8271465, 0.6780883, ..."
4,cafcd9496c808085,0.0284096 /m/04dr76w 0.5452898 0.0000000 0.623...,"[[0.0284096, /m/04dr76w, 0.5452898, 0.0000000,..."


In [11]:
def dets_to_xywh(dets):
    res = []
    for det in dets:
        conf = float(det[0])
        c1 = det[1]
        xmin = float(det[2])
        ymin = float(det[3])
        xmax = float(det[4])
        ymax = float(det[5])
        c2 = det[6]
        new_det = [(xmin+xmax)/2, (ymin+ymax)/2, xmax-xmin, ymax-ymin, ','.join([c1, c2]), conf]
        res.append(new_det)
    return res                                                                 

In [12]:
dets1 = df1.dets.map(lambda x: dets_to_xywh(x))
dets2 = df2.dets.map(lambda x: dets_to_xywh(x))
dets3 = df3.dets.map(lambda x: dets_to_xywh(x))

In [13]:
len(dets3)

99999

In [14]:
all_dets = list(zip(dets1, dets2, dets3))

In [20]:
list(all_dets[0])

[[[0.4225872,
   0.7855249,
   0.09809100000000004,
   0.11465199999999998,
   '/m/04bcr3,/m/083vt',
   0.230295],
  [0.4164936,
   0.8415807,
   0.09220319999999999,
   0.12517220000000007,
   '/m/04bcr3,/m/083vt',
   0.1540681],
  [0.2644319,
   0.80169145,
   0.4960746,
   0.38901129999999995,
   '/m/04bcr3,/m/083vt',
   0.1465968],
  [0.37917725,
   0.8223401,
   0.3286859,
   0.33334339999999996,
   '/m/04bcr3,/m/083vt',
   0.139526],
  [0.5228503,
   0.8293957000000001,
   0.41957979999999995,
   0.33469059999999995,
   '/m/04bcr3,/m/083vt',
   0.1192753],
  [0.4355377,
   0.8051736,
   0.12684060000000003,
   0.18564020000000003,
   '/m/04bcr3,/m/083vt',
   0.0940905],
  [0.0471477,
   0.7997306,
   0.0942954,
   0.16852539999999994,
   '/m/04bcr3,/m/083vt',
   0.0908246],
  [0.488379,
   0.7854983,
   0.2336702,
   0.19535619999999998,
   '/m/04bcr3,/m/083vt',
   0.0804716],
  [0.3888574,
   0.64703055,
   0.23984680000000003,
   0.5091243000000001,
   '/m/04bcr3,/m/083vt',
   

In [21]:
# new paralle ensemble
from multiprocessing import Pool
import time

bg = time.time()

with Pool(50) as p:
    final_ens = p.map(GeneralEnsemble, all_dets)
    
print('total time:', time.time()-bg)

total time: 244.28011846542358


In [61]:
final_ens = []
for i in tqdm(range(len(dets1))):
    ens = GeneralEnsemble([dets1[i], dets2[i], dets3[i]])
    final_ens.append(ens)


100%|██████████| 99999/99999 [23:40<00:00, 70.41it/s]  


In [22]:
def xywh_to_pre_str(dets):
    res = []
    if len(dets) < 1:
        return ''
    for det in dets:
        conf = det[5]
        c1, c2 = det[4].split(',')
        xmin = det[0] - (det[2] / 2)
        xmax = det[0] + (det[2] / 2)
        ymin = det[1] - (det[3] / 2)
        ymax = det[1] + (det[3] / 2)
        res.append('{:.7f}'.format(conf))
        res.append(c1)
        res.append('{:.7f}'.format(xmin))
        res.append('{:.7f}'.format(ymin))
        res.append('{:.7f}'.format(xmax))
        res.append('{:.7f}'.format(ymax))

        res.append(c2)
        res.append('{:.7f}'.format(xmin))
        res.append('{:.7f}'.format(ymin))
        res.append('{:.7f}'.format(xmax))
        res.append('{:.7f}'.format(ymax))
        res.append('is')

    res = [str(x) for x in res]
    return ' '.join(res)

In [23]:
pred_strings = pd.Series(final_ens).map(lambda x: xywh_to_pre_str(x)).values
len(pred_strings)

99999

In [24]:
df_sub = pd.read_csv('sub_is_42/sub_retina_is_42_0802_1_lb06401.csv')
df_sub.PredictionString = pred_strings
df_sub.head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.1037687 /m/04bcr3 0.3684591 0.7268276 0.4784...
1,763c72626bf24534,0.6366923 /m/04bcr3 0.6903263 0.8243049 0.9988...
2,7540c162180973b1,0.0189767 /m/04dr76w 0.2606493 0.2071633 0.301...
3,c4e522dce52cb02f,0.5760854 /m/04bcr3 0.8312336 0.6764226 0.9919...
4,cafcd9496c808085,0.0094699 /m/04dr76w 0.5452898 0.0000000 0.623...


In [25]:
df_sub.to_csv('ensemble_is_42_0824.csv', columns=['ImageId', 'PredictionString'], index=False)

In [26]:
!kaggle competitions submit -c open-images-2019-visual-relationship -f ensemble_is_42_0824.csv -m "submit"

100%|██████████████████████████████████████| 1.66G/1.66G [00:52<00:00, 34.3MB/s]
Successfully submitted to Open Images 2019 - Visual Relationship

In [27]:
len(final_ens[100])

142

In [76]:
pd.Series([[1],[2]]).map(lambda x: x + ['a'])

0    [1, a]
1    [2, a]
dtype: object

In [15]:
""" 
Ensembling methods for object detection.
"""

""" 
General Ensemble - find overlapping boxes of the same class and average their positions
while adding their confidences. Can weigh different detectors with different weights.
No real learning here, although the weights and iou_thresh can be optimized.

Input: 
 - dets : List of detections. Each detection is all the output from one detector, and
          should be a list of boxes, where each box should be on the format 
          [box_x, box_y, box_w, box_h, class, confidence] where box_x and box_y 
          are the center coordinates, box_w and box_h are width and height resp.
          The values should be floats, except the class which should be an integer.

 - iou_thresh: Threshold in terms of IOU where two boxes are considered the same, 
               if they also belong to the same class.
               
 - weights: A list of weights, describing how much more some detectors should
            be trusted compared to others. The list should be as long as the
            number of detections. If this is set to None, then all detectors
            will be considered equally reliable. The sum of weights does not
            necessarily have to be 1.

Output:
    A list of boxes, on the same format as the input. Confidences are in range 0-1.
"""
def GeneralEnsemble(dets, iou_thresh = 0.5, weights=None):
    assert(type(iou_thresh) == float)
    
    ndets = len(dets)
    
    if weights is None:
        w = 1/float(ndets)
        weights = [w]*ndets
    else:
        assert(len(weights) == ndets)
        
        s = sum(weights)
        for i in range(0, len(weights)):
            weights[i] /= s

    out = list()
    used = list()
    
    for idet in range(0,ndets):
        det = dets[idet]
        for box in det:
            if box in used:
                continue
                
            used.append(box)
            # Search the other detectors for overlapping box of same class
            found = []
            for iodet in range(0, ndets):
                odet = dets[iodet]
                
                if odet == det:
                    continue
                
                bestbox = None
                bestiou = iou_thresh
                for obox in odet:
                    if not obox in used:
                        # Not already used
                        if box[4] == obox[4]:
                            # Same class
                            iou = computeIOU(box, obox)
                            if iou > bestiou:
                                bestiou = iou
                                bestbox = obox
                                
                if not bestbox is None:
                    w = weights[iodet]
                    found.append((bestbox,w))
                    used.append(bestbox)
                            
            # Now we've gone through all other detectors
            if len(found) == 0:
                new_box = list(box)
                new_box[5] /= ndets
                out.append(new_box)
            else:
                allboxes = [(box, weights[idet])]
                allboxes.extend(found)
                
                xc = 0.0
                yc = 0.0
                bw = 0.0
                bh = 0.0
                conf = 0.0
                
                wsum = 0.0
                for bb in allboxes:
                    w = bb[1]
                    wsum += w

                    b = bb[0]
                    xc += w*b[0]
                    yc += w*b[1]
                    bw += w*b[2]
                    bh += w*b[3]
                    conf += w*b[5]
                
                xc /= wsum
                yc /= wsum
                bw /= wsum
                bh /= wsum    

                new_box = [xc, yc, bw, bh, box[4], conf]
                out.append(new_box)
    return out
    
def getCoords(box):
    x1 = float(box[0]) - float(box[2])/2
    x2 = float(box[0]) + float(box[2])/2
    y1 = float(box[1]) - float(box[3])/2
    y2 = float(box[1]) + float(box[3])/2
    return x1, x2, y1, y2
    
def computeIOU(box1, box2):
    x11, x12, y11, y12 = getCoords(box1)
    x21, x22, y21, y22 = getCoords(box2)
    
    x_left   = max(x11, x21)
    y_top    = max(y11, y21)
    x_right  = min(x12, x22)
    y_bottom = min(y12, y22)

    if x_right < x_left or y_bottom < y_top:
        return 0.0    
        
    intersect_area = (x_right - x_left) * (y_bottom - y_top)
    box1_area = (x12 - x11) * (y12 - y11)
    box2_area = (x22 - x21) * (y22 - y21)        
    
    iou = intersect_area / (box1_area + box2_area - intersect_area)
    return iou
    
if __name__=="__main__":
    # Toy example
    dets = [ 
            [[0.1, 0.1, 1.0, 1.0, 1, 0.9], [1.2, 1.4, 0.5, 1.5, 1, 0.9]],
            [[0.2, 0.1, 0.9, 1.1, 1, 0.8],[1.19, 1.4, 0.5, 1.5, 1, 0.9]],
            [[5.0,5.0,1.0,1.0,1,0.5]]
           ]
    
    ens = GeneralEnsemble(dets, weights = [1.0, 0.1, 0.5])
    print(ens)

[[0.10909090909090909, 0.1, 0.990909090909091, 1.009090909090909, 1, 0.6125], [1.199090909090909, 1.4000000000000001, 0.5, 1.5, 1, 0.61875], [5.0, 5.0, 1.0, 1.0, 1, 0.16666666666666666]]


In [ ]:
[[0.10909090909090909, 0.1, 0.990909090909091, 1.009090909090909, 'a', 0.6125], [1.2, 1.4, 0.5, 1.5, 'a', 0.3], [5.0, 5.0, 1.0, 1.0, 'a', 0.16666666666666666]]